In [1]:
from pycalphad import Database, Model, variables as v
from pycalphad import equilibrium
import numpy as np
from pycalphad.tests.datasets import *
dbf = Database(ROSE_TDB)

In [2]:
eq2 = equilibrium(dbf, ['H', 'HE', 'LI'], ['TEST'],
                 {v.X('LI'): 0.3, v.MU('H'): -7385.549841, v.NP('TEST'): 1e-6,
                  v.P: 1e5}, verbose=True, calc_opts={'pdens': 10})
print(eq2)

Components: H HE LI
Phases: TEST 


AttributeError: 'StateVariable' object has no attribute 'default_value'